In [ ]:
from astropy.io import fits
import numpy as np
import os

# See the BACKGROUND LEVELLING CELL on the START.ipynb script for a description of the functions

def bg_level(ref, path, save_path):
    
    # Creates the savepath in case it does not exist
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    # Encodes the folder so it can be iterated
    folder = os.fsencode(path)
    
    # Iterates over the files in the folder
    for item in os.listdir(folder):
        
        # Decodes the current item so it becomes a string
        item = os.fsdecode(item)
        
        # To have the full path of the item
        item = path + item
        
        # Opens the .fits item and reads its header
        hdu = fits.open(item)
        header = hdu[0].header
        
        # If the current .fits does not correspond to the detector which will be used for reference, then pass
        if (header['DETECTOR'] != ref):
            pass
        
        else:
            
            # Stores the data from the .fits file
            im = hdu[0].data
            
            # The median value of the image (the mean value would be affected by saturated pixels)
            median = np.nanmedian(im)
            
            # Now iterates over the other files in the folder
            for scnditem in os.listdir(folder):
                
                # Decodes the current scnditem so it becomes a string
                scnditem = os.fsdecode(scnditem)
                
                # The name of the current file
                name = scnditem
                
                # To have the full path of the scnditem
                scnditem = path + scnditem
                
                # Opens the .fits item and reads its header
                hdu2 = fits.open(scnditem)
                scndheader = hdu2[0].header
                
                # If the scnditem is the item then copy it to the savepath since it is the detector which all other detectors will be levelled to
                if (scndheader['DETECTOR'] == ref):
                    os.system("cp " + scnditem + " " + save_path)
                    pass
                
                # If the scnditem corresponds to the same filter
                elif (scndheader['FILTER01'] == header['FILTER01'] and scndheader['DATE-OBS'] == header['DATE-OBS'] and scndheader['DATA-TYP'] == 'OBJECT'):
                    
                    # Stores the data from the second .fits file
                    im2 = hdu2[0].data
                    
                    # The median value of the second image (the mean value would be affected by saturated pixels)
                    median2 = np.nanmedian(im2)
                    
                    # Levels the second image to the image of reference
                    im2 = im2 * median / median2
                    
                    # Saves the new levelled image into the savepath folder
                    hduimfinal = fits.PrimaryHDU(data = im2, header = scndheader)
                    hduimfinal.writeto(save_path + name, overwrite = True)

def bg_level_2(ref, path, save_path):
    
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    folder = os.fsencode(path)
    
    for item in os.listdir(folder):
        
        item = os.fsdecode(item) #Decodes the current item so it becomes a string
        
        item = path + item #To comply with the func() function parameter
        
        hdu = fits.open(item)
        header = hdu[0].header
        
        if ref not in item:
            pass
        
        else:
            
            im = hdu[0].data
            
            median = np.nanmedian(im)
            
            for scnditem in os.listdir(folder):
                
                scnditem = os.fsdecode(scnditem)
                
                name = scnditem
                
                scnditem = path + scnditem
                
                hdu2 = fits.open(scnditem)
                scndheader = hdu2[0].header
                
                im2 = hdu2[0].data
                
                median2 = np.nanmedian(im2)
                
                im2 = im2 * median / median2
                
                hduimfinal = fits.PrimaryHDU(data = im2, header = scndheader)
                hduimfinal.writeto(save_path + name, overwrite = True)